# ProfessorAI
Ask questions and get answers based on lecture content.

In [ ]:
import ipywidgets as widgets
import langchain
import pinecone
import openai
import tiktoken
import os

In [ ]:
# IMPORTANT Only run this if you want to set your keys using environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
PINECONE_API = os.environ.get("PINECONE_API_KEY")
PINCONE_ENV = os.environ.get("PINECONE_API_ENV")

In [ ]:
pinecone.init(
    api_key=PINECONE_API,
    environment=PINCONE_ENV
)

: 